## 🪄 Install `raga-testing-platform` library


In [ ]:
pip install raga-testing-platform -U

### Put your `ACCESS KEY & SECRET KEY`


In [2]:
# Define the access key, secret key, and host
ACCESS_KEY = "Put access key"
SECRET_KEY = "Put secret key"
HOST = "https://backend.platform.raga.ai"

### Import All raga lib from raga module


In [3]:
from raga import *
import datetime

# **Image Recognition**


### 🪄 Set up a Run


In [4]:
run_name = "colab-check"

## Active Learning Test


In [ ]:
dataset_name = "philips_emb"
budget = 1150

test_session = TestSession(
    project_name="MRI Classification",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)

edge_case_detection = active_learning(test_session=test_session,
                                      dataset_name = dataset_name,
                                      test_name = "active_learniing1",
                                      type = "active_learning",
                                      output_type="curated_dataset",
                                      embed_col_name="ImageVectorsM1",
                                      budget=budget)

test_session.add(edge_case_detection)

test_session.run()

# Class Imbalance Test


In [ ]:
dataset_name = "philips_emb"
metric_threshold = 0.1
rules = ClassImbalanceRules()

rules.add(metric="js_divergence", ideal_distribution="uniform", metric_threshold=metric_threshold, label="ALL")
rules.add(metric="chi_squared_test", ideal_distribution="uniform", metric_threshold=metric_threshold, label="ALL")


test_session = TestSession(
    project_name="MRI Classification",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)

distribution_test = class_imbalance_test(test_session=test_session,
                                         dataset_name=dataset_name,
                                         test_name="Class Imbalance Test",
                                         type="class_imbalance",
                                         output_type="image_classification",
                                         annotation_column_name="GroundTruth",
                                         rules=rules)

test_session.add(distribution_test)
test_session.run()

# Near Duplicates Test


In [ ]:
dataset_name = "philips_emb"
metric_threshold = 0.965

test_session = TestSession(
    project_name="MRI Classification",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)

rules = LQRules()
rules.add(metric="similarity_score", metric_threshold=metric_threshold)



edge_case_detection = nearest_duplicate(test_session=test_session,
                                          dataset_name = dataset_name,
                                          test_name = "Near-Duplicates-v4",
                                          type = "near_duplicates",
                                          output_type="near_duplicates",
                                          embed_col_name="ImageVectorsM1",
                                          rules=rules)

test_session.add(edge_case_detection)

test_session.run()

# Outlier Detection Test


In [ ]:
threshold=33
dataset_name = "philips_emb"

test_session = TestSession(
    project_name="MRI Classification",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)


rules = DriftDetectionRules()
rules.add(type="anomaly_detection", dist_metric="Mahalanobis", _class="ALL", threshold=threshold)
edge_case_detection = data_drift_detection(test_session=test_session,
                                           test_name="Drift-detection-test-1",
                                           dataset_name=dataset_name,
                                           embed_col_name = "ImageVectorsM1",
#                                            output_type = "image_classification",
                                           output_type = "outlier_detection",
                                           rules = rules)
test_session.add(edge_case_detection)

test_session.run()

# Data Leakage Test


In [ ]:
reference_dataset_name = "philips_emb_good"
eval_dataset_name = "philips_emb_bad"

metric_threshold = 0.95

test_session = TestSession(
    project_name="MRI Classification",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)

rules = LQRules()
rules.add(metric = 'overlapping_samples', metric_threshold = metric_threshold)


edge_case_detection = data_leakage_test(test_session=test_session,
                                           test_name="dataleakage1",
                                           train_dataset_name=reference_dataset_name,
                                           dataset_name=eval_dataset_name,
                                           type = "data_leakage",
                                           output_type="image_data",
                                           train_embed_col_name="ImageVectorsM1",
                                           embed_col_name = "ImageVectorsM1",
                                           rules = rules)

test_session.add(edge_case_detection)

test_session.run()

# Labeling Quality Test


In [ ]:
dataset_name = 'philips_custom_emb'
metric_threshold = 0.2


test_session = TestSession(
    project_name="MRI Classification",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)


rules = LQRules()
rules.add(metric="mistake_score", label=["ALL"], metric_threshold=metric_threshold)
edge_case_detection = labelling_quality_test(
    test_session=test_session,
    dataset_name=dataset_name,
    test_name="IC- Labeling Quality Test",
    type="labelling_consistency",
    output_type="image_classification",
    mistake_score_col_name="MistakeScore",
    embedding_col_name="ImageVectorsM1",
    rules=rules,
)

test_session.add(edge_case_detection)
test_session.run()

# Scenario imbalance


In [ ]:
dataset_name = "philips_custom_emb" 

rules = SBRules()
rules.add(metric="js_divergence", ideal_distribution="uniform", metric_threshold=0.1)
rules.add(metric="chi_squared_test", ideal_distribution="uniform", metric_threshold=0.1)


# clustering is required only at cluster level
cls_default = clustering(test_session=test_session,
                         dataset_name=dataset_name,
                         method="k-means",
                         embedding_col="ImageVectorsM1",
                         level="image",
                         args={"numOfClusters": 5}
                         )


edge_case_detection = scenario_imbalance(test_session=test_session,
                                            dataset_name = dataset_name,
                                            test_name = "Scenario_Imbalance",
                                            type = "scenario_imbalance",
                                            output_type="cluster",
                                            embedding= "ImageVectorsM1",
                                            rules = rules,
                                            clustering = cls_default
                                             )
test_session.add(edge_case_detection)

test_session.run()